In [1]:
import os,json
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.utils.data_utils import get_file
from keras.preprocessing import image
from keras.optimizers import SGD, RMSprop, Adam
from keras import backend as K
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
import matplotlib.pyplot as plt
K.set_image_data_format('channels_last')
plt.style.use('ggplot')
%matplotlib inline

Using TensorFlow backend.


In [2]:
from keras.applications.inception_v3 import InceptionV3,preprocess_input
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(200,200,3))

In [3]:
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200, 200, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 99, 99, 32)    864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 99, 99, 32)    96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 99, 99, 32)    0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [4]:
for layer in base_model.layers:
    layer.trainable =False
X = base_model.output
X = GlobalAveragePooling2D()(X)
X = Dense(activation='softmax', units=5)(X)
model = Model(inputs=base_model.input, outputs=X)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200, 200, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 99, 99, 32)    864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 99, 99, 32)    96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 99, 99, 32)    0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [5]:
path = 'data/flower_photos'
batches = image.ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    path, target_size=(200,200), class_mode='categorical', batch_size=16, shuffle=True)
assert batches.num_class==5
steps_per_epoch = batches.samples // batches.batch_size
model.fit_generator(
    batches, 
    steps_per_epoch=steps_per_epoch, 
    epochs=1)

Found 3670 images belonging to 5 classes.
Epoch 1/1
229/229 [==============================] - 767s - loss: 0.9887 - acc: 0.6307   
